In [2]:
#import libraries
import mahotas
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.svm import SVC


In [ ]:
# os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
# physical_devices = tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# !pip install mahotas

In [ ]:
# https://www.kaggle.com/datasets/alxmamaev/flowers-recognition?resource=download

In [3]:
# location holding the directories with images 
data_path = r'C:\Users\rajbo\Downloads\flowers'
# create empty lists to hold the images being read


In [4]:
x_image = []
y_label = []

labels = os.listdir(data_path)
#read images and load into lists for image and labels(directory name)
#images sized to fixed 300 x 300 for consistency across images.

In [5]:
labels

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [6]:
for dirname in labels:
    filepath = os.path.join(data_path, dirname)
    for file in os.listdir(filepath):
        filename = os.path.join(filepath, file)
#         print(filename)
        image = cv2.imread(filename)
        image = cv2.resize(image, (300,300))
        x_image.append(image.flatten())
        y_label.append(dirname)
        continue
    continue
    break 
x_image_n = np.array(x_image)
print(x_image_n.shape)
# (516, 270000)
y_label_n = np.array(y_label)
y_label_n.shape
# (516,)

(4317, 270000)


(4317,)

In [7]:
# split train and test data
x_train, x_test, y_train, y_test = train_test_split(x_image_n, y_label_n, test_size=0.2, random_state=0)

#################################
# (1) USING RAW PIXEL APPROACH ---DONT RUN
##################################

In [ ]:
#do not run 

In [ ]:

#train model with randomforestclassifer
# clf_rfc = RandomForestClassifier(random_state=10).fit(x_train, y_train)


In [ ]:
# evaluate the model with test data
y_pred_rfc = clf_rfc.predict(x_test)
print(precision_score(y_test, y_pred_rfc, average='weighted'))
# 0.5047158190373734
print(accuracy_score(y_test, y_pred_rfc))  
# 0.47115384615384615

- #######################################################
## USING GLOBAL FEATURES for Image Classification
- ########################################################

In [21]:
x_image = []
y_label = []

for dirname in labels:
    filepath = os.path.join(data_path, dirname)
    for file in os.listdir(filepath):
        filename = os.path.join(filepath, file)
#         print(filename)
        image = cv2.imread(filename)
        image_resized = cv2.resize(image, (300,300))
        image_gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)
# HUMoments for shape
        image_hu = cv2.HuMoments(cv2.moments(image_gray)).flatten()
# Haralick for texture 
#         image_har = mahotas.features.haralick(image_gray).mean(axis=0)
         # convert the image to HSV color-space
        image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
# color histogram
        image_hist  = cv2.calcHist([image_hsv], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        cv2.normalize(image_hist, image_hist)
        image_hist_flat = image_hist.flatten()
# combine the features extracted
#         f_vector_concat = np.hstack([image_hist_flat, image_har, image_hu])
        f_vector_concat = np.hstack([image_hist_flat, image_hu])
#         print(f_vector_concat.shape)
        x_image.append(f_vector_concat)
        y_label.append(dirname)
        continue
    continue
    break

In [22]:
np.array(x_image).shape

(4317, 519)

In [23]:
# convert lists to numpy array
x_image_n = np.array(x_image)
y_label_n = np.array(y_label)

# feature scaling between 0-1 
scaler = MinMaxScaler(feature_range=(0, 1))
x_image_scaled = scaler.fit_transform(x_image_n)

#split data set
x_train, x_test, y_train, y_test = train_test_split(x_image_n, y_label_n, test_size=0.2, random_state=0)
#train the model


In [27]:
clf_rfc = RandomForestClassifier().fit(x_train, y_train)
# evaluate the model
y_pred_rfc = clf_rfc.predict(x_test)

### Traimning Accuracy

In [39]:
y_pred_train = clf_rfc.predict(x_train)
print(accuracy_score(y_train, y_pred_train))

0.9991311902693311


###  Test Accuracy

In [28]:
print(accuracy_score(y_test, y_pred_rfc))

0.6469907407407407


In [29]:
print(precision_score(y_test, y_pred_rfc, average='weighted', labels=np.unique(y_pred_rfc)))
# 0.5858423061196674


0.6493561979784127


In [30]:
print(confusion_matrix(y_test, y_pred_rfc))


[[ 95  33  10  13  12]
 [  9 152   5  25   6]
 [  6  19  66   3  45]
 [ 15  16   8 103   5]
 [ 11  27  20  17 143]]


In [31]:

print(f1_score(y_test, y_pred_rfc, average='weighted', labels=np.unique(y_pred_rfc)))
# 0.5648969666761506

0.6436306299998447


In [32]:
# https://www.linkedin.com/pulse/image-classification-opencv-scikit-learn-raj-ramanujam

In [36]:
clf_rfc.predict(x_test[0:4])

array(['tulip', 'dandelion', 'dandelion', 'daisy'], dtype='<U9')

In [37]:
y_test[0:4]

array(['sunflower', 'dandelion', 'dandelion', 'daisy'], dtype='<U9')